In [1]:
import sys
sys.path.append('../')
from mylibs import *

In [2]:
from data.data import Data
handler = Data()
data = handler.load('../artifacts_5s_128hz.pkl')
train_split, val_split, test_split = handler.split(data)

In [16]:
train_features, train_labels = handler.prepare_data(train_split, balance=True, lstm=True, dataset=False, ccn=True)

In [17]:
train_features.shape

TensorShape([11497, 640, 19, 1])

In [18]:
train_features = train_features[:2, :, :]
train_labels = train_labels[:2, :, :]

In [6]:
from architectures.BidirectionalLSTM import BidirectionalLSTM

In [7]:
model = BidirectionalLSTM(num_hidden_units=16, num_bidirectional_layers=1, num_conv_layers=1)

In [8]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy())

In [9]:
tf.expand_dims(train_features, axis=-1).shape

TensorShape([2, 640, 19, 1, 1])

In [10]:
model.fit(train_features, train_labels, epochs=1)

Tensor("bidirectional_lstm/Cast:0", shape=(None, 640, 19, 1), dtype=float32)
Tensor("bidirectional_lstm/time_distributed/Reshape_1:0", shape=(None, 640, 17, 8), dtype=float32)
Tensor("bidirectional_lstm/time_distributed_1/Reshape_1:0", shape=(None, 640, 8, 8), dtype=float32)
Tensor("bidirectional_lstm/Cast:0", shape=(None, 640, 19, 1), dtype=float32)
Tensor("bidirectional_lstm/time_distributed/Reshape_1:0", shape=(None, 640, 17, 8), dtype=float32)
Tensor("bidirectional_lstm/time_distributed_1/Reshape_1:0", shape=(None, 640, 8, 8), dtype=float32)
1/1 [==============================] - 9s 9s/step - loss: 0.6819


In [19]:
from architectures.MultiLayerLSTM import MultiLayerLSTM

In [20]:
model2 = MultiLayerLSTM(num_hidden_units=16, num_lstm_layers=1, num_conv_layers=1)

In [21]:
model2.compile(loss=tf.keras.losses.BinaryCrossentropy())

In [22]:
model2.fit(train_features, train_labels, epochs=1)

1/1 [==============================] - 6s 6s/step - loss: 0.6941


In [21]:
model = keras.models.Sequential()
model.add(layers.TimeDistributed(layers.Conv1D(filters=32, kernel_size=1, padding='same', activation='relu', input_shape=(640, 19))))
model.add(layers.Conv1D(filters=32, kernel_size=1, padding='same', activation='relu'))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(2, activation='softmax'))

In [23]:
model.build()
model.summary()

ValueError: You must provide an `input_shape` argument.

In [84]:
model = keras.models.Sequential()
model.add(layers.TimeDistributed(layers.Conv1D(filters=8, kernel_size=3, activation='relu'), input_shape=(640, 19,1))) # shape (timesteps, sequence length, features)
model.add(layers.TimeDistributed(layers.Conv1D(filters=16, kernel_size=3, activation='relu'))) # shape (timesteps, sequence length, features)
model.add(layers.TimeDistributed(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))) # shape (timesteps, sequence length, features)
model.add(layers.TimeDistributed(layers.MaxPool1D()))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.Dense(2))

model.compile(loss=tf.keras.losses.BinaryCrossentropy())


In [85]:
model.summary()

Model: "sequential_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_75 (TimeDi  (None, 640, 17, 8)       32        
 stributed)                                                      
                                                                 
 time_distributed_76 (TimeDi  (None, 640, 15, 16)      400       
 stributed)                                                      
                                                                 
 time_distributed_77 (TimeDi  (None, 640, 13, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_78 (TimeDi  (None, 640, 6, 32)       0         
 stributed)                                                      
                                                                 
 time_distributed_79 (TimeDi  (None, 640, 192)      

In [73]:
features = tf.expand_dims(train, axis=-1)
features.shape

TensorShape([11497, 640, 19, 1])

In [87]:
model.fit(features[:2,:,:,:], labels[:2,:,:], epochs=1)

1/1 [==============================] - 7s 7s/step - loss: 0.4100
